# Sentinel-2 Melt Pond Classification

This program executes melt pond classification on imagery files processed from the previous scripts. The program outputs an HDF (.h5) file which will then be processed in the "S2_Analysis" script which will assess the numerical breakdown of each classification element. The HDF files may be converted as a .tif file and instructions are provided in the "Useful_Assets" script. 

In [ ]:
import rasterio
from rasterio import mask
import fiona
from pylab import *
import numpy as np
import sys, os
import pandas as pd
import h5py
from PIL import Image
import csv

## The 'peakdet' Function

The 'peakdet' function is used to set up histogram thresholds necessary to make the pixel-by-pixel classification. It is provided below to aid the classification methods. 

In [ ]:
def peakdet(v, delta, x = None):
    
    maxtab = []
    mintab = []
       
    if x is None:
        x = np.arange(len(v))
    
    v = np.asarray(v)
    
    if len(v) != len(x):
        sys.exit('Input vectors v and x must have same length')
    
    if not np.isscalar(delta):
        sys.exit('Input argument delta must be a scalar')
    
    if delta <= 0:
        sys.exit('Input argument delta must be positive')
    
    mn, mx = np.Inf, np.NINF
    mnpos, mxpos = np.nan, np.nan
    
    lookformax = True
   # mintab.append((0, 0.))
    
    for i in np.arange(len(v)):
        this = v[i]
        if this > mx:
            mx = this
            mxpos = x[i]
        if this < mn:
            mn = this
            mnpos = x[i]
        
        if lookformax:
            if this < mx-delta:
                maxtab.append((mxpos, mx))
                mn = this
                mnpos = x[i]
                lookformax = False
        else:
            if this > mn+delta:
                mintab.append((mnpos, mn))
                mx = this
                mxpos = x[i]
                lookformax = True
    return np.array(maxtab), np.array(mintab)

## Image Classification

The image classification procedures are provided in one cell with in-line comments to aid users. The classification steps are briefly as follows:

1. Reading in land masks to remove land pixels.
2. Removing image border pixels.
3. Seperating pixels between water and **not** water.
4. Determining ice pixels.
5. Seperating open water and melt pond pixels.
6. Combining results into an HDF output.

In [ ]:
# 1. Reading in land masks to remove land pixels.

# The 'landmask' directory provides shapefiles of Sentinel-2 tiles with land regions. Please take note of where
# this directory is located in your local machine and make appropriate adjustments to the script. The tiles with
# land pixels have been organized in a csv file named 'land_tiles.csv' and the script is designed to make appropriate
# adjustments to files whose pathname includes these tiles. 

# Read the tile list for tiles with land in them. 
# Please make sure to change the file path below!
land_tiles=pd.read_csv('/Volumes/Jaemin_IceDrive/MeltPonds/MeltEvolution/TestAssets/land_tiles.csv').tiles.values

direc='/Volumes/Jaemin_IceDrive/MeltPonds/MeltEvolution/TestAssets/'

# HDF Outputs will be saved in a seperate folder.
if (os.path.exists(direc+'classification_hdf/')==False):
    os.mkdir(direc+'classification_hdf/')

    
#store results
f1=open(direc+'results.txt','a')
# log errors
log=open(direc+'errorlog.txt','a')


# The following variable is the path of the base directory where the imagery files are located.
# Please make sure to change the path based on your local machine. 
#direc='/Volumes/Jaemin_IceDrive/MeltPonds/MeltEvolution/TestAssets/'
B02 = direc+ 'B02/'

base_list = []
for item in os.listdir(B02):
    if ".jp2" in item:
        basefiles = item.split('_B02')[0]
        base_list.append(basefiles)

#print(baseitems)

#base_list = np.unique(baseitems)


for base in base_list:#[a[:-8] for a in os.listdir(direc+'B08/') if a.endswith('_B08.jp2')]:#['T08XNN_20200904T231129']:#T08XNH_20200930T210301']:#[a[:-8] for a in os.listdir(direc+'B08/') if a.endswith('_B08.jp2')][29:]:#['T14XMR_20200611T231131']: #['T20XNS_20200716T185921']:#[a[:-8] for a in os.listdir('.') if a.endswith('_B08.jp2')]:#[50:]:#['T14XNS_20200625T225121']:#['T10XES_20200626T231129']:#'T12XWP_20200727T224109']:# 
    yblue=base+'_B02.jp2'
    ygreen=base+'_B03.jp2'
    yred=base+'_B04.jp2'
    ynir=base+'_B08.jp2'
    
    try:

        imblue = rasterio.open(direc+'B02/'+yblue)#rasterio.open(direc+'B02/yes/'+yblue)
        imgreen = rasterio.open(direc+'B03/'+ygreen)
        imred = rasterio.open(direc+'B04/'+yred)
        imnir = rasterio.open(direc+'B08/'+ynir)
        

        if base[:6] in land_tiles:
            print('applying land mask to ' + base)
            
            # read in the land mask that is in the appropriate reference
            with fiona.open(direc + "landmask/arcticocean326"+base[1:3]+".shp", "r") as shapefile:
                geoms = [feature["geometry"] for feature in shapefile]
            
            imblue_m = mask.mask(imblue, geoms, crop=True)
            imgreen_m = mask.mask(imgreen, geoms, crop=True)
            imred_m = mask.mask(imred, geoms, crop=True)
            imnir_m = mask.mask(imnir, geoms, crop=True)

            red=imred_m[0]
            green=imgreen_m[0]
            blue=imblue_m[0]
            nir=imnir_m[0]

            size_m=np.shape(red)
            
            sect_xmin,sect_xmax=0,size_m[1]
            sect_ymin,sect_ymax=0,size_m[2]
        else:
            red=imred.read(1)
            green=imgreen.read(1)
            blue=imblue.read(1)
            nir=imnir.read(1)
            
            size_m=np.shape(red)
            
            sect_xmin,sect_xmax=0,size_m[0]
            sect_ymin,sect_ymax=0,size_m[1]


        br1= np.divide((green.astype(float)-nir.astype(float)),(nir.astype(float)+green.astype(float)))

        red_c=np.array(red,copy= True)
        blue_c=np.array(blue,copy= True)
        green_c=np.array(green,copy= True)
        nir_c= np.array(nir,copy= True)

#2. Removing image border pixels.
#   Border Pixels are defined presently and subsequently removed later from the classification thresholds.
        border=red_c<500

#3. Seperating pixels between water and not water.
        bins=np.arange(-.5,.5,.02)
        Cbr1_n, bins= np.histogram(br1[sect_xmin:sect_xmax,sect_ymin:sect_ymax][~np.isnan(br1[sect_xmin:sect_xmax,sect_ymin:sect_ymax])&(~border[sect_xmin:sect_xmax,sect_ymin:sect_ymax])], bins)

        dx= 0.0005*sum(Cbr1_n)  # dx is 0.05% difference
        maxtab,mintab = peakdet(Cbr1_n,dx,x=None)

        if np.where(Cbr1_n==max(Cbr1_n))[0][0]==maxtab[0][0]: # if there is only one max- do the fwhm to determine the cut
            if maxtab[0,0]==0:
                loc_cut=0
            else:        
                fwhm= np.min(np.where((bins[:-1]>bins[int(maxtab[0,0])]) & (Cbr1_n<(maxtab[0,1]/2.)))) # smaller bin value than max_x, smaller coutn than .5 max_y
                loc_cut= int(maxtab[0,0]+ 2* (fwhm-maxtab[0,0])) # the cut location is 2* fwhm

        else: #otherwise it is the minimum
            loc_cut=int(mintab[np.where(maxtab[:,0]==np.where(Cbr1_n==max(Cbr1_n))[0][0])[0][0]-1,0])

        if len(mintab)>0:
            br1_cut =bins[int(mintab[-1,0])]
        else:
            br1_cut=bins[loc_cut]
        water_mask=(br1>br1_cut)&(~border)


# 4. Determining ice pixels.
        binz=np.arange(0,10000,200)
        n,bins=np.histogram(red_c[sect_xmin:sect_xmax,sect_ymin:sect_ymax][(~border[sect_xmin:sect_xmax,sect_ymin:sect_ymax])&(~water_mask[sect_xmin:sect_xmax,sect_ymin:sect_ymax])].flatten(),bins=binz)
        dx= 0.0001*sum(n)  # dx is 0.01% difference
        maxtab,mintab = peakdet(n,dx,x=None)
        #print (maxtab, mintab)

        if np.where(n==max(n))[0][0]==maxtab[0][0]: # if there is only one max or first max is highest
            if maxtab[0,0]==0:
                bin_cut=0
            else:      
                fwhm= np.max(np.where((bins[:-1]<bins[int(maxtab[0,0])]) & (n<(maxtab[0,1]/2.)))) # smaller bin value than max_x, smaller coutn than .5 max_y
                bin_cut= int(maxtab[0,0]+ 2* (fwhm-maxtab[0,0])) # the cut location is 2* fwhm

        else:
            bin_cut=int(mintab[np.where(maxtab[:,0]==np.where(n==max(n))[0][0])[0][0]-1,0])

        ice_cut=bins[bin_cut]

        ice_mask=(red_c>ice_cut)&(~border)&(~water_mask)
        other_mask=(red_c<=ice_cut)&(~border)&(~water_mask)


#5. Seperating open water and melt pond pixels.
        binz=np.arange(0,10000,200)
    
        n,bins=np.histogram(blue_c[(~ice_mask)&(~border)&(~other_mask)].flatten(),bins=binz)
        dx= 0.0001*sum(n)  # dx is 0.01% difference
        maxtab,mintab = peakdet(n,dx,x=None)

        if (len(maxtab)>1):
            if bins[int(mintab[-1,0])]>2000:
                ow_cut=bins[int(mintab[-1,0])]
            else:
                n, bins= np.histogram(blue_c[water_mask], bins)
                fwhm= np.min(np.where((bins[:-1]>bins[int(maxtab[-1,0])]) & (n<(maxtab[-1,1]/2.))))
                loc_cut= int(maxtab[-1,0]+ 4* (fwhm-maxtab[-1,0]))
                ow_cut=bins[loc_cut]

        elif bins[int(maxtab[0,0])]<4000:
            n, bins= np.histogram(blue_c[water_mask], bins)
            fwhm= np.min(np.where((bins[:-1]>bins[int(maxtab[0,0])]) & (n<(maxtab[0,1]/2.))))
            loc_cut= int(maxtab[0,0]+ 4* (fwhm-maxtab[0,0]))
            ow_cut=bins[loc_cut]
        else:
            ow_cut=4000

        ow_mask=(blue_c<ow_cut)&(water_mask)
        mp_mask=(blue_c>=ow_cut)&(water_mask)

# 6. Combining results into an HDF output.
   # Before results can be exported, we want to take summary statistics of each defined class.
        ow_pix=np.sum(ow_mask)
        mp_pix=np.sum(mp_mask)
        ice_pix=np.sum(ice_mask)
        border_pix=np.sum(border)
        other_pix=np.sum(other_mask)
        im_pix= (sect_xmax-sect_xmin)*(sect_ymax-sect_ymin)

        f1.write(base+'\t'+str(im_pix)+'\t'+str(border_pix)+'\t'+str(ice_pix)+'\t'+str(ow_pix)+'\t'+str(mp_pix)+'\t'+str(other_pix)+'\n')


        # Check sum for pixels
        # If pixels don't sum to 1, this means that there was some error or misclassification.
        pixel_sum = (1 * border + 1 * ice_mask + 1 * ow_mask + 1 * mp_mask + 1 * other_mask)
        if (~np.all(pixel_sum == 1)):
            log.write("Sum pixels not 1 {0}: {1}\n".format(str(base), str(e)))

        # Calculated Parameters
        MPF=np.nan
        SIC=np.round((mp_pix+ice_pix)/np.float32(ice_pix+mp_pix+ow_pix)*100,2)
        if SIC>15:
            MPF=np.round((mp_pix)/np.float32(ice_pix+mp_pix)*100,2)

        #Set up dataset for hdf5 files
        classification = np.zeros_like(border, dtype=np.int8)    # 8 bit int
        classification += (1 * ice_mask + 2 * ow_mask + 3 * mp_mask + 4 * other_mask)

        fout_name = direc+'classification_hdf/{}_classification.h5'.format(base)
        with h5py.File(fout_name, "w") as fout:
            fout.attrs["title"] = "Classification of Sentinel-2 Sea Ice Summer Melt Features"
            fout.attrs["creator_email"] = "buckley@umd.edu"

            fout.attrs["references"] = "Buckley, E. M., Farrell, S. L., Herzfeld, U. C., Webster, M., Trantow, T., Baney, O. N., Duncan, K., Han, H., Lawson, M. (2023). Observing the Evolution of Summer Melt on Multiyear Sea Ice with ICESat-2 and Sentinel-2 [Manuscript in Preparation]."
            fout.attrs["source_image"] = base
            fout.attrs["MPF (%)"] = MPF
            fout.attrs["SIC (%)"] = SIC


            dset = fout.create_dataset("classification", data=classification,
                    compression="gzip")
            dset.attrs["categories"] = "0 border, 1 ice, 2 open water, 3 melt pond, 4 other"
        
               
        
    except Exception as e:
        log.write("Failed to process {0}: {1}\n".format(str(base), str(e)))
    finally:
        pass
    
f1.close()
log.close()